In [1]:
import requests
import math
import pandas as pd

# Stats globale par pays

cf page 25 rapport OST sur les publis 2005-2018

In [23]:
global_data = []
for year in range(2005, 2023):
    print(year, end=',')
    r = requests.get(f'https://api.openalex.org/works?filter=publication_year:{year}&group-by=institutions.country_code').json()['group_by']
    for ix, e in enumerate(r):
        e['year'] = year
        e['rank'] = ix
    global_data += r


2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,

In [33]:
df_global = pd.DataFrame(global_data)
top_country = df_global[df_global.year==2021].head(10).key_display_name.tolist()
df_global_top = df_global[df_global.key_display_name.apply(lambda x:x in top_country)]
pd.pivot_table(df_global_top, index='year', columns="key_display_name", values='rank')

key_display_name,Brazil,China,France,Germany,India,Indonesia,Japan,United Kingdom of Great Britain and Northern Ireland,United States of America,unknown
year,,,,,,,,,,
2005,11,2,6,5,12,50,4,3,1,0
2006,11,2,6,5,13,50,4,3,1,0
2007,11,2,5,6,12,46,4,3,1,0
2008,10,2,5,4,12,45,6,3,1,0
2009,10,2,5,4,12,44,6,3,1,0
2010,10,2,5,4,12,39,6,3,1,0
2011,9,2,5,4,11,23,6,3,1,0
2012,8,2,5,4,9,25,6,3,1,0
2013,8,2,4,5,7,24,6,3,1,0


# Exploration sur une requête (affiliation)

In [2]:
random_size = 1000
first_year = 2016

In [3]:
BASE_URL = f"https://api.openalex.org/works?sample={random_size}&seed=0&filter=publication_year:{first_year}-"

In [4]:


user_input = 'beta cnrs'
#user_input='LIP6'
#user_input = 'institut porc'
#user_input = 'BNP'
#user_input = 'Danone'
#user_input = 'philip morris'
collab_fr = False

#user_input = 'russia'
#collab_fr = True



In [5]:
url = BASE_URL + f",raw_affiliation_string.search:{user_input}"
if collab_fr:
    url += f",raw_affiliation_string.search:france"
print(url)

https://api.openalex.org/works?sample=1000&seed=0&filter=publication_year:2016-,raw_affiliation_string.search:beta cnrs


In [6]:
def get_data(url):
    data = []
    res = requests.get(url).json()
    nb_data = res['meta']['count']
    print(nb_data)
    nb_page = math.ceil(nb_data / res['meta']['per_page'])
    data = res['results']
    for p in range(2, nb_page + 1):
        print(p, end=',')
        res = requests.get(url+f"&page={p}").json()
        data += res['results']
    assert(len(data) == nb_data)
    return data
        

In [7]:
x = get_data(url)
df = pd.DataFrame(x)

385
2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,

# Nombre de publications par an

In [8]:
df.groupby('publication_year').id.count()

publication_year
2016    51
2017    43
2018    41
2019    61
2020    59
2021    77
2022    49
2023     4
Name: id, dtype: int64

# Publications par domaines et par an

In [9]:
concepts_year = []
for p in x:
    publication_year = p['publication_year']
    has_concept = False
    concepts = p.get('concepts')
    sorted(concepts, key = lambda e: e['score'])
    for c in p.get('concepts'):
        if c.get('level') == 0:
            elt = {'publication_year': publication_year}
            elt.update(c)
            concepts_year.append(elt)
            break # on ne garde que le premier concept avec le score le plus élevé
    if has_concept is False:
        concepts_year.append({'publication_year': publication_year, 'display_name': 'N/A'})
        
df_concept = pd.DataFrame(concepts_year)
pd.pivot_table(df_concept, columns=['display_name'], 
               index='publication_year', values='id', aggfunc='count', margins=True).fillna(0)

display_name,Art,Biology,Business,Chemistry,Computer science,Economics,Engineering,Environmental science,Geography,Geology,History,Mathematics,Medicine,N/A,Philosophy,Political science,Psychology,Sociology,All
publication_year,,,,,,,,,,,,,,,,,,,
2016,1.0,1.0,12.0,2.0,3.0,20.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,2.0,51
2017,0.0,2.0,7.0,0.0,0.0,20.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,3.0,6.0,1.0,1.0,43
2018,0.0,0.0,11.0,0.0,2.0,21.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,41
2019,1.0,4.0,13.0,0.0,4.0,28.0,0.0,3.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,0.0,61
2020,0.0,3.0,12.0,0.0,3.0,30.0,0.0,1.0,4.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,59
2021,0.0,0.0,24.0,0.0,6.0,24.0,0.0,4.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,7.0,4.0,3.0,77
2022,0.0,0.0,14.0,0.0,1.0,17.0,1.0,4.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,1.0,1.0,49
2023,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
All,2.0,10.0,96.0,2.0,19.0,161.0,1.0,16.0,17.0,1.0,1.0,7.0,4.0,0.0,5.0,23.0,13.0,7.0,385


# Top revues

In [10]:
df.host_venue.apply(lambda x: x['display_name'] ).value_counts().head(20)

RePEc: Research Papers in Economics                           50
Social Science Research Network                               17
HAL (Le Centre pour la Communication Scientifique Directe)    11
Applied Economics                                             10
SpringerBriefs in well-being and quality of life research      9
India studies in business and economics                        8
Management international                                       8
Ecological Economics                                           8
Springer International Publishing eBooks                       7
Research Policy                                                7
Environmental Modeling & Assessment                            6
Journal of Evolutionary Economics                              6
European Journal of Law and Economics                          5
Journal of Public Economic Theory                              5
Frontiers in economic history                                  5
Resource and Energy Econo

# Top auteurs

In [11]:
def input_in_string(input_str, aff_str):
    if aff_str is None:
        return False
    for a in input_str.lower().split(' '):
        if a not in aff_str.lower():
            return False
    return True

In [12]:
auteurs_data_input = []
auteurs_data_fr = []
for p in x:
    for a in p.get('authorships'):
        if input_in_string(user_input, a.get('raw_affiliation_string')):
            elt = {'doi': p['doi'], 'raw_affiliation_string': a['raw_affiliation_string']}
            elt.update(a['author'])
            auteurs_data_input.append(elt)
            break
        if input_in_string('france', a.get('raw_affiliation_string')):
            elt = {'doi': p['doi'], 'raw_affiliation_string': a['raw_affiliation_string']}
            elt.update(a['author'])
            auteurs_data_fr.append(elt)
            break
print('auteurs input')
print()
print(pd.DataFrame(auteurs_data_input).display_name.value_counts().head(10))
print()
print('---')
print('auteur fr hors input')
print()
print(pd.DataFrame(auteurs_data_fr).display_name.value_counts().head(10))

auteurs input

Claude Diebolt             28
Olivier Damette            16
Blaise Gnimassoun          10
Francis L. Munier          10
Marielle Brunette           7
Phu Nguyen-Van              7
Kene Boun My                7
Emilie Ruiz                 6
Véronique Schaeffer         5
Thierry Burger-Helmchen     5
Name: display_name, dtype: int64

---
auteur fr hors input

Stefano Bosi           15
Dramane Coulibaly       3
Benjamin Ouvrard        3
Véronique Schaeffer     2
Giuseppe Attanasi       2
Ngoc Minh Pham          2
Valentin Bellassen      2
Patrice Loisel          2
Nicolas Pinsard         2
Chouaib Jouf            2
Name: display_name, dtype: int64


# Top partenaires - données brutes

In [13]:
partenaires_data_brut = []
signature_data = []
known_keys = [] # on ne compte qu'une fois un couple publi x affiliation qu'il y ait 1 ou 20 auteurs
for p in x:
    for a in p.get('authorships'):
        for i in a['raw_affiliation_string'].strip().split(';'):
            if i and p['doi']:
                elt_key = p['doi']+';'+i
                elt = {'doi': p['doi'], 'raw_affiliation_string': i.strip()}
                if elt_key in known_keys:
                    continue
                else:
                    known_keys.append(elt_key)
                if input_in_string(user_input, i):
                    signature_data.append(elt)
                else:
                    partenaires_data_brut.append(elt)
                    
                
            
pd.DataFrame(partenaires_data_brut).raw_affiliation_string.value_counts().head(10)

Forward College, Université de Neuchâtel, Neuchâtel, Switzerland                               9
Southampton Business School, University of Southampton, Southampton, UK                        8
(Business School, University of Roehampton, London, UK)                                        8
Department for VET and Skills, Cedefop, Thessaloniki, Greece                                   7
University of Angers - Bureau of Economic Theory and Application (BETA)                        4
Groupe d'Analyse et de Théorie Economique (GATE), CNRS, University of Lyon                     4
Toulouse School of Economics, INRA, University of Toulouse Capitole, Toulouse, France.         3
Universal Scientific Education and Research Network (USERN)                                    3
Europäische Kommission, Joint Research Centre (JRC), Ispra, Italien                            3
Centre d'Economie et de Sociologie Rurales Appliquées à l'Agriculture et aux Espaces Ruraux    3
Name: raw_affiliation_string, 

# Top partenaires - données normalisées par OpenAlex

In [14]:
partenaires_data = []
known_keys = []
for p in x:
    for a in p.get('authorships'):
        if user_input.lower() not in a.get('raw_affiliation_string').lower():
            for i in a.get('institutions'):
            #for i in a['raw_affiliation_string'].strip().split(';'):
                if p.get('doi') and i.get('display_name'):
                    elt_key = p['doi']+';'+i['display_name']
                    elt = {'doi': p['doi'], 'display_name': i['display_name'], 'country_code': i['country_code'], 
                          'concepts': p['concepts'] }
                    if elt_key not in known_keys:
                        partenaires_data.append(elt)
                        known_keys.append(elt_key)

df_partenaires_data = pd.DataFrame(partenaires_data)
print('partenaires france')
print()
print(df_partenaires_data[df_partenaires_data.country_code=='FR'].display_name.value_counts().head(10))
print()
print('-----')
print('partenaires étranger')
print()
print(df_partenaires_data[df_partenaires_data.country_code!='FR'].display_name.value_counts().head(10))

partenaires france

University of Lorraine                                       86
University of Strasbourg                                     80
Agro ParisTech                                               15
Bureau for Economic Theory and Applications                  14
Paris Dauphine University                                    12
University of Paris-Saclay                                   11
Paris West University Nanterre La Défense                     8
Groupe d'Analyse et de Théorie Economique Lyon St Etienne     6
Paris School of Economics                                     6
University of Upper Alsace                                    6
Name: display_name, dtype: int64

-----
partenaires étranger

Beta                                                                                                       61
University of Neuchâtel                                                                                     9
University of Southampton                                 

# Top pays partenaires

In [15]:
df_partenaires_data.country_code.value_counts().head(10)

FR    392
IT     89
GB     49
ES     40
DE     25
US     25
CH     17
NL     16
DK     16
VN     13
Name: country_code, dtype: int64

# Variété des signatures de l'input

In [16]:
pd.DataFrame(signature_data).raw_affiliation_string.value_counts().head(20)

Faculté des Sciences Economiques et de Gestion BETA-CNRS, Université de Strasbourg, Strasbourg, France      9
BETA/CNRS, Faculty of Economics, University of Strasbourg, Strasbourg, France                               9
Université de Strasbourg, Université de Lorraine, CNRS, BETA, Strasbourg, France                            8
Faculty of Economics, BETA/CNRS, Strasbourg, France                                                         8
Université de Lorraine, Université de Strasbourg, AgroParisTech, CNRS, INRA, BETA, 54000, Nancy, France     7
Université de Lorraine, Université de Strasbourg, AgroParisTech, CNRS, INRAE, BETA, Nancy, France           7
Université de Lorraine, Université de Strasbourg, CNRS, BETA, Nancy, France.                                6
Université de Strasbourg, Université de Lorraine, CNRS, BETA, 67000 Strasbourg, France                      4
University of Lorraine, University of Strasbourg, AgroParisTech, CNRS, INRA, BETA Nancy France              4
BETA/CNRS 

# Thématiques quand collab avec un pays

In [17]:
partner = 'FR'

In [18]:
concepts_with_partner = []
for row in df_partenaires_data[df_partenaires_data.country_code==partner].itertuples():
    current_concepts =  [c for c in row.concepts if c['level'] > 0]
    for e in current_concepts:
        e['doi'] = row.doi
    concepts_with_partner += current_concepts
pd.DataFrame(concepts_with_partner).display_name.value_counts().head(10)

Ecology                       96
Law                           74
Microeconomics                59
Natural resource economics    58
Finance                       53
Econometrics                  52
Macroeconomics                49
Archaeology                   35
Marketing                     35
Environmental economics       31
Name: display_name, dtype: int64

In [19]:
#pd.DataFrame(concepts_with_partner)